## 玖零猴的Demo

In [24]:
import os
import numpy as np
import SimpleITK as sitk

In [25]:
flair_name = "_flair.nii.gz"
t1_name = "_t1.nii.gz"
t1ce_name = "_t1ce.nii.gz"
t2_name = "_t2.nii.gz"
mask_name = "_seg.nii.gz"

In [26]:
#bratshgg_path = r"D:\Project\CollegeDesign\dataset\2-MICCAI_BraTS_2018\MICCAI_BraTS_2018_Data_Training\HGG"
bratshgg_path = r"C:\Users\user\Desktop\MICCAI_BraTS_2019_Data_Training\MICCAI_BraTS_2019_Data_Training\HGG"

In [27]:
#bratslgg_path = r"D:\Project\CollegeDesign\dataset\2-MICCAI_BraTS_2018\MICCAI_BraTS_2018_Data_Training\LGG"
bratslgg_path = r"C:\Users\user\Desktop\MICCAI_BraTS_2019_Data_Training\MICCAI_BraTS_2019_Data_Training\LGG"

In [28]:
outputImg_path = r".\trainImage"

In [29]:
outputMask_path = r".\trainMask"

In [30]:
if not os.path.exists(outputImg_path):
    os.mkdir(outputImg_path)
if not os.path.exists(outputMask_path):
    os.mkdir(outputMask_path)

In [31]:
def file_name_path(file_dir, dir=True, file=False):
    """
    get root path,sub_dirs,all_sub_files
    :param file_dir:
    :return: dir or file
    """
    for root, dirs, files in os.walk(file_dir):
        if len(dirs) and dir:
            print("sub_dirs:", dirs)
            return dirs
        if len(files) and file:
            print("files:", files)
            return files

In [32]:
pathhgg_list = file_name_path(bratshgg_path)
pathlgg_list = file_name_path(bratslgg_path)

sub_dirs: ['BraTS19_2013_10_1', 'BraTS19_2013_11_1', 'BraTS19_2013_12_1', 'BraTS19_2013_13_1', 'BraTS19_2013_14_1', 'BraTS19_2013_17_1', 'BraTS19_2013_18_1', 'BraTS19_2013_19_1', 'BraTS19_2013_20_1', 'BraTS19_2013_21_1', 'BraTS19_2013_22_1', 'BraTS19_2013_23_1', 'BraTS19_2013_25_1', 'BraTS19_2013_26_1', 'BraTS19_2013_27_1', 'BraTS19_2013_2_1', 'BraTS19_2013_3_1', 'BraTS19_2013_4_1', 'BraTS19_2013_5_1', 'BraTS19_2013_7_1', 'BraTS19_CBICA_AAB_1', 'BraTS19_CBICA_AAG_1', 'BraTS19_CBICA_AAL_1', 'BraTS19_CBICA_AAP_1', 'BraTS19_CBICA_ABB_1', 'BraTS19_CBICA_ABE_1', 'BraTS19_CBICA_ABM_1', 'BraTS19_CBICA_ABN_1', 'BraTS19_CBICA_ABO_1', 'BraTS19_CBICA_ABY_1', 'BraTS19_CBICA_ALN_1', 'BraTS19_CBICA_ALU_1', 'BraTS19_CBICA_ALX_1', 'BraTS19_CBICA_AME_1', 'BraTS19_CBICA_AMH_1', 'BraTS19_CBICA_ANG_1', 'BraTS19_CBICA_ANI_1', 'BraTS19_CBICA_ANP_1', 'BraTS19_CBICA_ANV_1', 'BraTS19_CBICA_ANZ_1', 'BraTS19_CBICA_AOC_1', 'BraTS19_CBICA_AOD_1', 'BraTS19_CBICA_AOH_1', 'BraTS19_CBICA_AOO_1', 'BraTS19_CBICA_AOP_1',

In [33]:
def normalize(slice, bottom=99, down=1):
    """
    normalize image with mean and std for regionnonzero,and clip the value into range
    :param slice:
    :param bottom:
    :param down:
    :return:
    """
    #有点像“去掉最低分去掉最高分”的意思,使得数据集更加“公平”
    b = np.percentile(slice, bottom)
    t = np.percentile(slice, down)
    slice = np.clip(slice, t, b)#限定范围numpy.clip(a, a_min, a_max, out=None)

    #除了黑色背景外的区域要进行标准化
    image_nonzero = slice[np.nonzero(slice)]
    if np.std(slice) == 0 or np.std(image_nonzero) == 0:
        return slice
    else:
        tmp = (slice - np.mean(image_nonzero)) / np.std(image_nonzero)
        # since the range of intensities is between 0 and 5000 ,
        # the min in the normalized slice corresponds to 0 intensity in unnormalized slice
        # the min is replaced with -9 just to keep track of 0 intensities
        # so that we can discard those intensities afterwards when sampling random patches
        tmp[tmp == tmp.min()] = -9 #黑色背景区域
        return tmp

np.percentil
分位数:
在我们中学就有过中位数，其实中位数就是一个二分位数，取中位数左边区间的值的概率等于取其右边区间的值的概率。另外在大学的概率论课程中，另一个比较常见的是四分位数，也就是25%,50%,75%这三个切分点。
分位数意义：表示了在这个样本集中从小至大排列之后小于某值的样本子集占总样本集的比例
我采用了99与1这两个分位数来作为鉴定outliers分布的界限从而进行异常值的修正。这样子的修正可以以更好的准确率来得到我们理想的数据集。
有点像去掉最低分去掉最高分的意思

In [34]:
def crop_ceter(img,croph,cropw):   
    #for n_slice in range(img.shape[0]):
    height,width = img[0].shape 
    starth = height//2-(croph//2)
    startw = width//2-(cropw//2)        
    return img[:,starth:starth+croph,startw:startw+cropw]

In [35]:
for subsetindex in range(len(pathhgg_list)):
    brats_subset_path = bratshgg_path + "/" + str(pathhgg_list[subsetindex]) + "/"
    #获取每个病例的四个模态及Mask的路径
    flair_image = brats_subset_path + str(pathhgg_list[subsetindex]) + flair_name
    t1_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t1_name
    t1ce_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t1ce_name
    t2_image = brats_subset_path + str(pathhgg_list[subsetindex]) + t2_name
    mask_image = brats_subset_path + str(pathhgg_list[subsetindex]) + mask_name
    #获取每个病例的四个模态及Mask数据
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    t1_src = sitk.ReadImage(t1_image, sitk.sitkInt16)
    t1ce_src = sitk.ReadImage(t1ce_image, sitk.sitkInt16)
    t2_src = sitk.ReadImage(t2_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    #GetArrayFromImage()可用于将SimpleITK对象转换为ndarray
    flair_array = sitk.GetArrayFromImage(flair_src)
    t1_array = sitk.GetArrayFromImage(t1_src)
    t1ce_array = sitk.GetArrayFromImage(t1ce_src)
    t2_array = sitk.GetArrayFromImage(t2_src)
    mask_array = sitk.GetArrayFromImage(mask)
    #对四个模态分别进行标准化,由于它们对比度不同
    flair_array_nor = normalize(flair_array)
    t1_array_nor = normalize(t1_array)
    t1ce_array_nor = normalize(t1ce_array)
    t2_array_nor = normalize(t2_array)
    #裁剪(偶数才行)
    flair_crop = crop_ceter(flair_array_nor,160,160)
    t1_crop = crop_ceter(t1_array_nor,160,160)
    t1ce_crop = crop_ceter(t1ce_array_nor,160,160)
    t2_crop = crop_ceter(t2_array_nor,160,160)
    mask_crop = crop_ceter(mask_array,160,160) 
    print(str(pathhgg_list[subsetindex]))
    #切片处理,并去掉没有病灶的切片
    for n_slice in range(flair_crop.shape[0]):
        if np.max(mask_crop[n_slice,:,:]) != 0:
            maskImg = mask_crop[n_slice,:,:]
            
            FourModelImageArray = np.zeros((flair_crop.shape[1],flair_crop.shape[2],4),np.float)
            flairImg = flair_crop[n_slice,:,:]
            flairImg = flairImg.astype(np.float)
            FourModelImageArray[:,:,0] = flairImg
            t1Img = t1_crop[n_slice,:,:]
            t1Img = t1Img.astype(np.float)
            FourModelImageArray[:,:,1] = t1Img
            t1ceImg = t1ce_crop[n_slice,:,:]
            t1ceImg = t1ceImg.astype(np.float)
            FourModelImageArray[:,:,2] = t1ceImg
            t2Img = t2_crop[n_slice,:,:]
            t2Img = t2Img.astype(np.float)
            FourModelImageArray[:,:,3] = t2Img       
        
            imagepath = outputImg_path + "\\" + str(pathhgg_list[subsetindex]) + "_" + str(n_slice) + ".npy"
            maskpath = outputMask_path + "\\" + str(pathhgg_list[subsetindex]) + "_" + str(n_slice) + ".npy"
            np.save(imagepath,FourModelImageArray)#(160,160,4) np.float dtype('float64')
            np.save(maskpath,maskImg)# (160, 160) dtype('uint8') 值为0 1 2 4
print("Done！")
        
    

BraTS19_2013_10_1
BraTS19_2013_11_1
BraTS19_2013_12_1
BraTS19_2013_13_1
BraTS19_2013_14_1
BraTS19_2013_17_1
BraTS19_2013_18_1
BraTS19_2013_19_1
BraTS19_2013_20_1
BraTS19_2013_21_1
BraTS19_2013_22_1
BraTS19_2013_23_1
BraTS19_2013_25_1
BraTS19_2013_26_1
BraTS19_2013_27_1
BraTS19_2013_2_1
BraTS19_2013_3_1
BraTS19_2013_4_1
BraTS19_2013_5_1
BraTS19_2013_7_1
BraTS19_CBICA_AAB_1
BraTS19_CBICA_AAG_1
BraTS19_CBICA_AAL_1
BraTS19_CBICA_AAP_1
BraTS19_CBICA_ABB_1
BraTS19_CBICA_ABE_1
BraTS19_CBICA_ABM_1
BraTS19_CBICA_ABN_1
BraTS19_CBICA_ABO_1
BraTS19_CBICA_ABY_1
BraTS19_CBICA_ALN_1
BraTS19_CBICA_ALU_1
BraTS19_CBICA_ALX_1
BraTS19_CBICA_AME_1
BraTS19_CBICA_AMH_1
BraTS19_CBICA_ANG_1
BraTS19_CBICA_ANI_1
BraTS19_CBICA_ANP_1
BraTS19_CBICA_ANV_1
BraTS19_CBICA_ANZ_1
BraTS19_CBICA_AOC_1
BraTS19_CBICA_AOD_1
BraTS19_CBICA_AOH_1
BraTS19_CBICA_AOO_1
BraTS19_CBICA_AOP_1
BraTS19_CBICA_AOS_1
BraTS19_CBICA_AOZ_1
BraTS19_CBICA_APK_1
BraTS19_CBICA_APR_1
BraTS19_CBICA_APY_1
BraTS19_CBICA_APZ_1
BraTS19_CBICA_AQA_1
BraTS

In [36]:
for subsetindex in range(len(pathlgg_list)):
    brats_subset_path = bratslgg_path + "/" + str(pathlgg_list[subsetindex]) + "/"
    #获取每个病例的四个模态及Mask的路径
    flair_image = brats_subset_path + str(pathlgg_list[subsetindex]) + flair_name
    t1_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t1_name
    t1ce_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t1ce_name
    t2_image = brats_subset_path + str(pathlgg_list[subsetindex]) + t2_name
    mask_image = brats_subset_path + str(pathlgg_list[subsetindex]) + mask_name
    #获取每个病例的四个模态及Mask数据
    flair_src = sitk.ReadImage(flair_image, sitk.sitkInt16)
    t1_src = sitk.ReadImage(t1_image, sitk.sitkInt16)
    t1ce_src = sitk.ReadImage(t1ce_image, sitk.sitkInt16)
    t2_src = sitk.ReadImage(t2_image, sitk.sitkInt16)
    mask = sitk.ReadImage(mask_image, sitk.sitkUInt8)
    #GetArrayFromImage()可用于将SimpleITK对象转换为ndarray
    flair_array = sitk.GetArrayFromImage(flair_src)
    t1_array = sitk.GetArrayFromImage(t1_src)
    t1ce_array = sitk.GetArrayFromImage(t1ce_src)
    t2_array = sitk.GetArrayFromImage(t2_src)
    mask_array = sitk.GetArrayFromImage(mask)
    #对四个模态分别进行标准化,由于它们对比度不同
    flair_array_nor = normalize(flair_array)
    t1_array_nor = normalize(t1_array)
    t1ce_array_nor = normalize(t1ce_array)
    t2_array_nor = normalize(t2_array)
    #裁剪(偶数才行)
    flair_crop = crop_ceter(flair_array_nor,160,160)
    t1_crop = crop_ceter(t1_array_nor,160,160)
    t1ce_crop = crop_ceter(t1ce_array_nor,160,160)
    t2_crop = crop_ceter(t2_array_nor,160,160)
    mask_crop = crop_ceter(mask_array,160,160) 
    print(str(pathlgg_list[subsetindex]))
    #切片处理,并去掉没有病灶的切片
    for n_slice in range(flair_crop.shape[0]):
        if np.max(mask_crop[n_slice,:,:]) != 0:
            maskImg = mask_crop[n_slice,:,:]
            
            FourModelImageArray = np.zeros((flair_crop.shape[1],flair_crop.shape[2],4),np.float)
            flairImg = flair_crop[n_slice,:,:]
            flairImg = flairImg.astype(np.float)
            FourModelImageArray[:,:,0] = flairImg
            t1Img = t1_crop[n_slice,:,:]
            t1Img = t1Img.astype(np.float)
            FourModelImageArray[:,:,1] = t1Img
            t1ceImg = t1ce_crop[n_slice,:,:]
            t1ceImg = t1ceImg.astype(np.float)
            FourModelImageArray[:,:,2] = t1ceImg
            t2Img = t2_crop[n_slice,:,:]
            t2Img = t2Img.astype(np.float)
            FourModelImageArray[:,:,3] = t2Img       
        
            imagepath = outputImg_path + "\\" + str(pathlgg_list[subsetindex]) + "_" + str(n_slice) + ".npy"
            maskpath = outputMask_path + "\\" + str(pathlgg_list[subsetindex]) + "_" + str(n_slice) + ".npy"
            np.save(imagepath,FourModelImageArray)#(160,160,4) np.float dtype('float64')
            np.save(maskpath,maskImg)# (160, 160) dtype('uint8') 值为0 1 2 4
print("Done!")

BraTS19_2013_0_1
BraTS19_2013_15_1
BraTS19_2013_16_1
BraTS19_2013_1_1
BraTS19_2013_24_1
BraTS19_2013_28_1
BraTS19_2013_29_1
BraTS19_2013_6_1
BraTS19_2013_8_1
BraTS19_2013_9_1
BraTS19_TCIA09_141_1
BraTS19_TCIA09_177_1
BraTS19_TCIA09_254_1
BraTS19_TCIA09_255_1
BraTS19_TCIA09_312_1
BraTS19_TCIA09_402_1
BraTS19_TCIA09_428_1
BraTS19_TCIA09_451_1
BraTS19_TCIA09_462_1
BraTS19_TCIA09_493_1
BraTS19_TCIA09_620_1
BraTS19_TCIA10_103_1
BraTS19_TCIA10_109_1
BraTS19_TCIA10_130_1
BraTS19_TCIA10_152_1
BraTS19_TCIA10_175_1
BraTS19_TCIA10_202_1
BraTS19_TCIA10_241_1
BraTS19_TCIA10_261_1
BraTS19_TCIA10_266_1
BraTS19_TCIA10_276_1
BraTS19_TCIA10_282_1
BraTS19_TCIA10_299_1
BraTS19_TCIA10_307_1
BraTS19_TCIA10_310_1
BraTS19_TCIA10_325_1
BraTS19_TCIA10_330_1
BraTS19_TCIA10_346_1
BraTS19_TCIA10_351_1
BraTS19_TCIA10_387_1
BraTS19_TCIA10_393_1
BraTS19_TCIA10_408_1
BraTS19_TCIA10_410_1
BraTS19_TCIA10_413_1
BraTS19_TCIA10_420_1
BraTS19_TCIA10_442_1
BraTS19_TCIA10_449_1
BraTS19_TCIA10_490_1
BraTS19_TCIA10_625_1
BraTS1